In [1]:
import torch
from transformers import AutoTokenizer
from src.transformers.models.bert import BertModel
import os
import numpy as np
import random
os.environ["CUDA_VISIBLE_DEVICES"]="0"


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained("zhihan1996/DNA_bert_6", trust_remote_code=True)
model = BertModel.from_pretrained("zhihan1996/DNA_bert_6", trust_remote_code=True).to(device)

/home/cameron/miniconda3/envs/cam_env_2/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
The argument `trust_remote_code` is to be used with Auto classes. It has no effect here and is ignored.
/home/cameron/miniconda3/envs/cam_env_2/lib/python3.11/site-packages/torch/_utils.py:830: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [2]:
from Bio import Entrez
from Bio import SeqIO
Entrez.email = "dufault77@gmail.com"
def download_sequence(accession, start, end):
    handle = Entrez.efetch(db="nucleotide", id=accession, rettype="fasta", strand=1, seq_start=start, seq_stop=end)
    seq_record = SeqIO.read(handle, "fasta")
    handle.close()
    return seq_record.seq.reverse_complement()

#sequence_prm1 = download_sequence('NC_000016.10', 11280831, 11281340)

#sequence_ = download_sequence('NC_000016.10', 11280831, 11281340)
#print(sequence_prm1)



In [3]:
(11281333+3) - (11280838-3)

501

In [4]:
region_mapping_list = []
region_type_mapping_list = []





mapping_dict = {}
mapping_dict_type = {}
mapping_dict[0] = "cls"
mapping_dict_type[0] = "cls"
sequence_prm1 = download_sequence('NC_000016.10', 11280841-3, 11281330+3)
for i in range(1, 195):
    mapping_dict[i] = "exon_1"
for i in range(195, 287):
    mapping_dict[i] = "intron_1"
for i in range(287, 490):
    mapping_dict[i] = "exon_2"
mapping_dict[490] = "sep"
for i in range(1, 195):
    mapping_dict_type[i] = "exon"
for i in range(195, 287):
    mapping_dict_type[i] = "intron"
for i in range(287, 490):
    mapping_dict_type[i] = "exon"
mapping_dict_type[490] = "sep"

region_mapping_list.append(mapping_dict)
region_type_mapping_list.append(mapping_dict_type)


mapping_dict = {}
mapping_dict_type = {}
mapping_dict[0] = "cls"
mapping_dict_type[0] = "cls"
sequence_1 = download_sequence("NC_000001.11",9424658-3, 9425159+3)
for i in range(1, 293):
    mapping_dict[i] = "ALU_1"
for i in range(293, 502):
    mapping_dict[i] = "MIR_1"
mapping_dict[502] = "sep"
for i in range(1, 293):
    mapping_dict_type[i] = "ALU"
for i in range(293, 502):
    mapping_dict_type[i] = "MIR"
mapping_dict_type[502] = "sep"
region_mapping_list.append(mapping_dict)
region_type_mapping_list.append(mapping_dict_type)


mapping_dict = {}
mapping_dict_type = {}
mapping_dict[0] = "cls"
mapping_dict_type[0] = "cls"
sequence_2 = download_sequence("NC_000001.11",9530501-3, 9530700+3)
for i in range(1, 64):
    mapping_dict[i] = "MIR_1"
for i in range(64, 200):
    mapping_dict[i] = "ALU_1"
mapping_dict[200] = "sep"
for i in range(1, 64):
    mapping_dict_type[i] = "ALU"
for i in range(64, 200):
    mapping_dict_type[i] = "MIR"
mapping_dict_type[200] = "sep"
region_mapping_list.append(mapping_dict)
region_type_mapping_list.append(mapping_dict_type)
#sequence_3
#sequence_4
#sequence_5
#sequence_6
#sequence_7
#sequence_8

In [5]:
num_layers = 12
num_heads = 12
skip = 3

In [6]:
tokens_dict = {"tokens": list()}
vocab_reverse = dict((value, key) for key, value in tokenizer.vocab.items())


attention_dict = dict()
point_position_dict = dict()
point_position_dict = dict()
agg_attn_dict = dict()
for layer in range(0,num_layers,skip):
    attention_dict[layer] = dict()
    point_position_dict[layer] = dict()
    for head in range(0,num_heads,skip):
        agg_attn_dict[f"{layer}_{head}"] = list()
        attention_dict[layer][head] = dict()
        attention_dict[layer][head]["layer"] = layer 
        attention_dict[layer][head]["head"] = head
        attention_dict[layer][head]["tokens"] = list() 
        point_position_dict[layer][head] = dict()
        point_position_dict[layer][head]["layer"] = layer 
        point_position_dict[layer][head]["head"] = head
        point_position_dict[layer][head]["tokens"] = list()
        point_position_dict[layer][head]["query"] = list()  
        point_position_dict[layer][head]["key"] = list()         

In [7]:
dataset = [str(sequence_prm1), str(sequence_1), str(sequence_2)]

sentence_stops = list()
sentence_starts = list()
pos = 0
for seq_num,sequence in enumerate(dataset):
    sentence_starts.append(pos)
    inputs = tokenizer(" ".join([sequence[i-3:i+3] for i in range(3, len(sequence)-4, 1)]), return_tensors = 'pt')
    out = model(**inputs.to(device), output_attentions=True, return_dict=True)
    tokens = [vocab_reverse[x] for x in inputs['input_ids'].tolist()[0]]   
    pos = pos+len(tokens)
    sentence_stops.append(pos) 
    for i,value in enumerate(tokens):
        single_token = {}
        single_token['value'] = value
        single_token['region_type'] = region_type_mapping_list[seq_num][i]
        single_token['region_name'] = region_mapping_list[seq_num][i]
        single_token['type'] = "query"
        single_token["length"] = len(tokens)
        single_token['pos_int'] = i
        single_token['position'] = i/(len(tokens)- 1)
        single_token['sentence'] = " ".join(tokens) 
   
        tokens_dict['tokens'].append(single_token)
        for layer in range(0,num_layers,skip): 
            for head in range(0,num_heads,skip):
                point_position_dict[layer][head]['query'].append(out.query[layer][head][i].detach().cpu().numpy())
                point_position_dict[layer][head]['key'].append(out.key[layer][head][i].detach().cpu().numpy())
                attention_dict[layer][head]['tokens'].append({'attention' : out.attentions[layer][0][head][i].detach().cpu().numpy()})
        

In [8]:
len(tokens)

201

In [9]:
# Getting point positions

#Some code from chatGPT!!!!!!!
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
import umap
import numpy as np
import tqdm

def get_pca_embeddings(vectors, n_components=2):
    pca = PCA(n_components=n_components)
    return pca.fit_transform(vectors)
def get_tsne_embeddings(vectors, n_components=2, perplexity=30.0):
    tsne = TSNE(n_components=n_components, perplexity=perplexity, n_iter=250)
    return tsne.fit_transform(vectors)
def get_umap_embeddings(vectors, n_components=2, n_neighbors=15, min_dist=0.1):
    umap_model = umap.UMAP(n_components=n_components, n_neighbors=n_neighbors, min_dist=min_dist)
    return umap_model.fit_transform(vectors)
def calculate_centroid(vectors):
    """Calculate the centroid of a list of vectors."""
    return np.mean(vectors, axis=0)
def translate_vectors(source_vectors, target_vectors):
    """Translate source_vectors so their centroid matches that of target_vectors."""
    source_centroid = calculate_centroid(source_vectors)
    target_centroid = calculate_centroid(target_vectors)
    translation = target_centroid - source_centroid
    translated_vectors = source_vectors + translation
    return translated_vectors

def calculate_norm(vector):
    return np.linalg.norm(vector)

for layer in tqdm.tqdm(range(0,num_layers,skip)): 
    for head in tqdm.tqdm(range(0,num_heads,skip)):
        translated_key = translate_vectors(point_position_dict[layer][head]['key'], point_position_dict[layer][head]['query'])
        vectors = np.stack(point_position_dict[layer][head]['query'] + [np.array(row) for row in translated_key])
        
        pca_2d = get_pca_embeddings(vectors, n_components=2)
        #pca_3d = get_pca_embeddings(vectors, n_components=3)

        tsne_2d = get_tsne_embeddings(vectors, n_components=2)
        #tsne_3d = get_tsne_embeddings(vectors, n_components=3)

        umap_2d = get_umap_embeddings(vectors, n_components=2)
        #umap_3d = get_umap_embeddings(vectors, n_components=3)
        
        for token in range(umap_2d.shape[0]):
            point_position_dict[layer][head]['tokens'].append({
                "tsne_x" : tsne_2d[token][0],
                "tsne_y" : tsne_2d[token][1],
                
                #"tsne_x_3d" : tsne_3d[token][0],
                #"tsne_y_3d" : tsne_3d[token][1],                
                #"tsne_z_3d" : tsne_3d[token][2],                
                
                "umap_x" : umap_2d[token][0],
                "umap_y" : umap_2d[token][1],          

                #"umap_x_3d" : umap_3d[token][0],
                #"umap_y_3d" : umap_3d[token][1],                
                #"umap_z_3d" : umap_3d[token][2],                         
                
                "pca_x" : pca_2d[token][0],
                "pca_y" : pca_2d[token][1],
                
                #"pca_x_3d" : pca_3d[token][0],
                #"pca_y_3d" : pca_3d[token][1],                                    
                #"pca_z_3d" : pca_3d[token][2],    
                
                "norm" : calculate_norm(vectors[token])                              
            })

100%|██████████| 4/4 [00:58<00:00, 14.51s/it]


In [10]:
for layer in tqdm.tqdm(range(0,num_layers,skip)): 
    for head in tqdm.tqdm(range(0,num_heads,skip)):
        h = [np.stack([token['attention'] for token in attention_dict[layer][head]['tokens'][sentence_starts[i]:sentence_stops[i]]]) for i in range(len(dataset))]
        max_h = max([x.shape[0] for x in h])
        shaped = [np.pad(x,(0,max_h - x.shape[0])) for x in h]
        avg =  np.average(np.stack(shaped),axis=0)
        agg_attn_dict[f"{layer}_{head}"] = [{"attention" : [round(x,5) for x in avg[i].tolist()]} for i in range(avg.shape[0])]
        
        



100%|██████████| 4/4 [00:00<00:00,  4.56it/s]


In [11]:
for layer in tqdm.tqdm(range(0,num_layers,skip)): 
    for head in tqdm.tqdm(range(0,num_heads,skip)):
        del point_position_dict[layer][head]["query"]
        del point_position_dict[layer][head]["key"]

100%|██████████| 4/4 [00:00<00:00, 103.22it/s]


In [12]:
for layer in tqdm.tqdm(range(0,num_layers,skip)): 
    for head in tqdm.tqdm(range(0,num_heads,skip)):
        for i in range(len(point_position_dict[layer][head]['tokens'])):
            for data_feature in ["tsne_x", "tsne_y", "umap_x", "umap_y", "norm", "pca_x", "pca_y"]:
                point_position_dict[layer][head]['tokens'][i][data_feature] = round(float(point_position_dict[layer][head]['tokens'][i][data_feature]),5)

100%|██████████| 4/4 [00:00<00:00, 35.79it/s]


In [13]:
import copy
tokens = copy.deepcopy(tokens_dict["tokens"])
pls = list()
for token in tqdm.tqdm(tokens):
    token["type"] = "key"
    tokens_dict["tokens"].append(token)
del tokens
    

100%|██████████| 1195/1195 [00:00<00:00, 6442407.81it/s]


In [14]:
for layer in tqdm.tqdm(range(0,num_layers,skip)): 
    for head in tqdm.tqdm(range(0,num_heads,skip)):
        for att in attention_dict[layer][head]['tokens']:
            att['attention'] = [round(x,5) for x in att['attention'].tolist()] 
        

100%|██████████| 4/4 [00:01<00:00,  2.13it/s]


## Writing to files

In [15]:
import json

In [16]:
with open("/home/cameron/repos/attention-viz-bio/web/data/DNABERT/agg_attn.json", "w") as fp:
    json.dump(agg_attn_dict , fp) 

In [17]:
del agg_attn_dict

In [18]:
with open("/home/cameron/repos/attention-viz-bio/web/data/DNABERT/tokens.json", "w") as fp:
    json.dump(tokens_dict , fp) 

In [19]:
del tokens_dict

In [20]:
for layer in tqdm.tqdm(range(0,num_layers,skip)): 
    for head in tqdm.tqdm(range(0,num_heads,skip)):
        with open(f"/home/cameron/repos/attention-viz-bio/web/data/DNABERT/attention/layer{layer}_head{head}.json", "w") as fp:
            json.dump(attention_dict[layer][head] , fp) 
        with open(f"/home/cameron/repos/attention-viz-bio/web/data/DNABERT/byLayerHead/layer{layer}_head{head}.json", "w") as fp:
            json.dump(point_position_dict[layer][head] , fp)             
        

100%|██████████| 4/4 [00:03<00:00,  1.24it/s]


## Writing aggregation things

In [69]:
tokens_dict_aggregate = {"tokens": list()}
vocab_reverse = dict((value, key) for key, value in tokenizer.vocab.items())


attention_dict_aggregate = dict()
point_position_dict_aggregate = dict()
agg_attn_dict_aggregate = dict()
for layer in range(0,num_layers,skip):
    attention_dict_aggregate[layer] = dict()
    point_position_dict_aggregate[layer] = dict()
    for head in range(0,num_heads,skip):
        agg_attn_dict_aggregate[f"{layer}_{head}"] = list()
        attention_dict_aggregate[layer][head] = dict()
        attention_dict_aggregate[layer][head]["layer"] = layer 
        attention_dict_aggregate[layer][head]["head"] = head
        attention_dict_aggregate[layer][head]["tokens"] = list() 
        point_position_dict_aggregate[layer][head] = dict()
        point_position_dict_aggregate[layer][head]["layer"] = layer 
        point_position_dict_aggregate[layer][head]["head"] = head
        point_position_dict_aggregate[layer][head]["tokens"] = list()  

In [70]:
original_list = list(region_mapping_list[0].values())
unique_list = []
for item in original_list:
    if item not in unique_list:
        unique_list.append(item)

In [71]:
def average_dicts(dict_list):
    if not dict_list:
        return {}
    
    # Initialize a dictionary to store the sum of values for each key
    sum_dict = {key: 0 for key in dict_list[0].keys()}

    # Sum up the values for each key
    for d in dict_list:
        for key in sum_dict.keys():
            sum_dict[key] += d[key]

    # Calculate the average for each key
    avg_dict = {key: value / len(dict_list) for key, value in sum_dict.items()}
    #avg_dict['norm'] = sum_dict['norm']

    return avg_dict

In [72]:

dataset = [str(sequence_prm1), str(sequence_1), str(sequence_2)]

sentence_stops_aggregate = list()
sentence_starts_aggregate = list()
pos = 0
for seq_num,sequence in enumerate(dataset):
    sentence_starts_aggregate.append(pos)    
    original_list = list(region_mapping_list[seq_num].values())
    tokens = []
    change_list = []
    for i,item in enumerate(original_list):
        if item not in tokens:
            change_list.append(i)
            tokens.append(item)
    change_list.append(len(original_list))
    pos = pos+len(tokens)
    sentence_stops_aggregate.append(pos) 
    for i,value in enumerate(tokens):
        single_token = {}
        single_token['value'] = value
        single_token['region_type'] = value.split("_")[0]
        single_token['region_name'] = value
        single_token['type'] = "query"
        single_token["length"] = len(tokens)
        single_token['pos_int'] = i
        single_token['position'] = i/(len(tokens)- 1)
        single_token['sentence'] = " ".join(tokens) 
   
        tokens_dict_aggregate['tokens'].append(single_token)
    for layer in range(0,num_layers,skip): 
        for head in range(0,num_heads,skip):       
            x =  [[attention_dict[layer][head]['tokens'][j+sentence_starts[seq_num]]['attention'][change_list[i]:change_list[i+1]] for i in range(len(tokens))] for j in range(len(original_list))  ]               
            out = [[np.average(attention_dict[layer][head]['tokens'][j+sentence_starts[seq_num]]['attention'][change_list[i]:change_list[i+1]]) for i in range(len(tokens))] for j in range(len(original_list))  ]
            for i in range(len(tokens)):
                point_position_dict_aggregate[layer][head]["tokens"].append(average_dicts(point_position_dict[layer][head]['tokens'][(sentence_starts[seq_num]+change_list[i]):(sentence_starts[seq_num]+change_list[i+1])]))
                attention_dict_aggregate[layer][head]['tokens'].append({'attention' : np.average(np.array(out[change_list[i]:change_list[i+1]]),axis=0)})
                

                
pos = 0
for seq_num,sequence in enumerate(dataset):
    original_list = list(region_mapping_list[seq_num].values())
    tokens = []
    change_list = []
    for i,item in enumerate(original_list):
        if item not in tokens:
            change_list.append(i)
            tokens.append(item)
    change_list.append(len(original_list))
    for layer in range(0,num_layers,skip): 
        for head in range(0,num_heads,skip):         
            for i in range(len(tokens)):
                point_position_dict_aggregate[layer][head]["tokens"].append(average_dicts(point_position_dict[layer][head]['tokens'][(sentence_stops[-1]+sentence_starts[seq_num]+change_list[i]):(sentence_stops[-1]+sentence_starts[seq_num]+change_list[i+1])]))

In [74]:
for layer in tqdm.tqdm(range(0,num_layers,skip)): 
    for head in tqdm.tqdm(range(0,num_heads,skip)):
        h = [np.stack([token['attention'] for token in attention_dict_aggregate[layer][head]['tokens'][sentence_starts_aggregate[i]:sentence_stops_aggregate[i]]]) for i in range(len(dataset))]
        max_h = max([x.shape[0] for x in h])
        shaped = [np.pad(x,(0,max_h - x.shape[0])) for x in h]
        avg =  np.average(np.stack(shaped),axis=0)
        agg_attn_dict_aggregate[f"{layer}_{head}"] = [{"attention" : [round(x,5) for x in avg[i].tolist()]} for i in range(avg.shape[0])]

100%|██████████| 4/4 [00:00<00:00, 361.78it/s]


In [75]:
for layer in tqdm.tqdm(range(0,num_layers,skip)): 
    for head in tqdm.tqdm(range(0,num_heads,skip)):
        for i in range(len(point_position_dict_aggregate[layer][head]['tokens'])):
            for data_feature in ["tsne_x", "tsne_y", "umap_x", "umap_y", "norm", "pca_x", "pca_y"]:
                point_position_dict_aggregate[layer][head]['tokens'][i][data_feature] = float(point_position_dict_aggregate[layer][head]['tokens'][i][data_feature])

100%|██████████| 4/4 [00:00<00:00, 523.27it/s]


In [76]:
import copy
tokens = copy.deepcopy(tokens_dict_aggregate["tokens"])
pls = list()
for token in tqdm.tqdm(tokens):
    token["type"] = "key"
    tokens_dict_aggregate["tokens"].append(token)
del tokens

100%|██████████| 13/13 [00:00<00:00, 235025.66it/s]


In [77]:
for layer in tqdm.tqdm(range(0,num_layers,skip)): 
    for head in tqdm.tqdm(range(0,num_heads,skip)):
        for att in attention_dict_aggregate[layer][head]['tokens']:
            att['attention'] = att['attention'].tolist() 

100%|██████████| 4/4 [00:00<00:00, 523.58it/s]


## Writing Aggregate to file

In [78]:
import json
with open("/home/cameron/repos/attention-viz-bio/web/data/DNABERT_agg/agg_attn.json", "w") as fp:
    json.dump(agg_attn_dict_aggregate , fp) 

In [79]:
with open("/home/cameron/repos/attention-viz-bio/web/data/DNABERT_agg/tokens.json", "w") as fp:
    json.dump(tokens_dict_aggregate , fp) 

In [80]:
## Writing to files
import json

for layer in tqdm.tqdm(range(0,num_layers,skip)): 
    for head in tqdm.tqdm(range(0,num_heads,skip)):
        with open(f"/home/cameron/repos/attention-viz-bio/web/data/DNABERT_agg/attention/layer{layer}_head{head}.json", "w") as fp:
            json.dump(attention_dict_aggregate[layer][head] , fp) 
        with open(f"/home/cameron/repos/attention-viz-bio/web/data/DNABERT_agg/byLayerHead/layer{layer}_head{head}.json", "w") as fp:
            json.dump(point_position_dict_aggregate[layer][head] , fp)             
        

100%|██████████| 4/4 [00:00<00:00, 245.17it/s]
